In [4]:
import pandas as pd
import numpy as np
import vnstock as vn

In [5]:
# Load data
START = "2019-01-01"
END = "2024-02-01"
def load_data(ticker):
        data = vn.stock_historical_data(ticker, START, END)    
        return data
data = load_data('VCB') 
data = data.set_index('time')
data

,open,high,low,close,volume,ticker
time,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB
2019-01-03,34820,34880,33980,34560,1071350,VCB
2019-01-04,34370,35080,33910,35080,1307310,VCB
2019-01-07,35660,35780,35140,35200,1175810,VCB
2019-01-08,35460,35590,35140,35460,1318810,VCB
...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB
2024-01-29,91800,92000,90900,91200,730500,VCB
2024-01-30,91200,91200,90100,91000,888600,VCB


# Volume

In [6]:
mean_20 = data['volume'].rolling(window=20).mean() #Tính trung bình 20 phiên gần nhất
data['on-balance_volume'] = data['volume']/mean_20 #Tính on-balance volume
data

,open,high,low,close,volume,ticker,on-balance_volume
time,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN
...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869


In [7]:
def label_values(a):
    if pd.isnull(a):
        return a
    elif a < 0.75:
        return 'low'
    elif 0.75 <= a <= 1.25:
        return 'medium'
    else:
        return 'high'
        
data['OBV_label'] = data['on-balance_volume'].apply(label_values)
data.sample(10)

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label
time,,,,,,,,
2019-12-10,55000,55380,54800,55060,512290,VCB,0.839411,medium
2023-02-06,78410,81540,77480,81290,1151200,VCB,0.969991,medium
2019-09-10,49780,51070,49710,50100,651950,VCB,1.061192,medium
2019-10-04,52870,53840,52740,53520,1121700,VCB,1.179420,medium
2021-05-19,62040,62040,61520,61710,1678100,VCB,0.835102,medium
2021-07-08,72930,74110,71480,72800,1384100,VCB,0.721616,low
2019-08-05,51070,51390,49710,49900,857570,VCB,0.925292,medium
2022-11-25,61900,62320,61640,61900,1675100,VCB,1.256762,high
2019-10-01,52930,53900,52740,53770,795100,VCB,0.941186,medium


# Bar

In [8]:
def compare_close_prices(t, t_minus_1):
    if t > t_minus_1:
        return 'up-bar'  
    else:
        return 'down-bar'

data['close_t_minus_1'] = data['close'].shift(1) #Giá đóng cửa ngày t-1

data['bar_type'] = data.apply(lambda row: compare_close_prices(row['close'], row['close_t_minus_1']), axis=1)
data.sample(10)

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type
time,,,,,,,,,,
2023-04-18,74770,75020,74090,74430,611900,VCB,1.117370,medium,74850.0,down-bar
2020-06-22,55290,55680,55290,55620,1308600,VCB,1.347742,high,55550.0,up-bar
2022-05-10,65280,67740,65280,67740,926000,VCB,0.724731,low,65960.0,up-bar
2022-08-10,67820,68590,67820,68160,507500,VCB,0.494774,low,68420.0,down-bar
2020-10-26,56920,57110,55940,55940,911390,VCB,0.938086,medium,56920.0,down-bar
2023-01-09,72650,73580,71800,73580,1667100,VCB,1.326279,high,71130.0,up-bar
2023-08-23,88900,88900,86100,86300,1754800,VCB,1.352619,high,88400.0,down-bar
2019-01-11,35590,35780,35400,35590,1115410,VCB,NaN,NaN,35460.0,up-bar
2021-03-25,62110,62960,61970,62370,1160300,VCB,0.927854,medium,62110.0,up-bar


# Spread

In [9]:
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para
time,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818


In [10]:
def label_spread(a):
    if 0 < a < 0.3: return 'low'
    elif 0.3 <= a <= 0.7: return 'medium'
    else: return 'high'

data['labe_spread'] = data['para'].apply(label_spread)
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,labe_spread
time,,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low


## Closing bar

In [22]:
def label_close_bar(data):
    q1 = data['low'] + (data['high'] - data['low'])/3
    q2 = data['low'] + 2*((data['high'] - data['low'])/3)
    if data['close'] < q1:
        return 'bottom-third'
    elif q1 <= data['close'] <= q2:
        return 'middle-third'
    else:
        return 'top-third' 

data['close_bar_label'] = data.apply(label_close_bar, axis=1)
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,labe_spread,close_bar_label
time,,,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium,top-third
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium,bottom-third
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low,top-third


In [21]:
data['close_bar_label'].values

array(['bottom-third', 'bottom-third', 'bottom-third', ...,
       'bottom-third', 'bottom-third', 'bottom-third'], dtype=object)